<a href="https://colab.research.google.com/github/shuv50/IBM_Data_Science/blob/main/SpaceX_WebScrapping_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Space X  Falcon 9 First Stage Landing Prediction

Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia

Here we will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled List of Falcon 9 and Falcon Heavy launches

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


In [ ]:
from IPython.display import display, HTML

# GIF URLs
gif_url_1 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif"
gif_url_2 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif"

# HTML to display GIFs side by side
html_code = f"""
<div style="display: flex; justify-content: center; gap: 20px; margin-top: 20px;">
    <img src="{gif_url_1}" alt="GIF 1" style="max-width: 100%; height: auto; border-radius: 10px;">
    <img src="{gif_url_2}" alt="GIF 2" style="max-width: 100%; height: auto; border-radius: 10px;">
</div>
"""

# Display the HTML
display(HTML(html_code))


More specifically, the launch records are stored in a HTML table shown below -

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


# Objectives

Web scrap Falcon 9 launch records with `BeautifulSoup`:
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame

# Install Packages

In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

# Import Libraries

In [ ]:
# Import libraries
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

# Dataframe settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#  Define Auxiliary Functions
Helper functions that will help us use the API to extract information using identification numbers in the launch data.

## Date/Time HTML data

In [ ]:
# Returns the data and time
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

## Booster Version HTML data

In [ ]:
# Returns the booster version
def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

## Landing Status data

In [ ]:
# Returns the landing status
def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out

def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = ' '.join(row.contents)

    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name

## Mass data

In [ ]:
# Returns the mass
def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

# Collect Data

## Wikipedia data

Let's request the HTML page from the below URL and get a response object.
<br>Perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.

In [ ]:
# Full data
spacex_wiki_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(spacex_wiki_url)

# Check the content of the response
# print(response.content)

## BeautifulSoup

Create a BeautifulSoup object from the HTML response
<br>[link text](https://)Use BeautifulSoup() to create a BeautifulSoup object from a response text content

In [ ]:
# HTML content
soup = BeautifulSoup(response.content, 'html.parser')
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

# Feature Selection

Next, we want to collect all relevant column names from the HTML table header
<br>Let's try to find all tables on the wiki page first.


Use the find_all function in the BeautifulSoup object, with element type `table`
<br>Starting from the third table is our target table contains the actual launch records.


Apply find_all() function with `th` element on first_launch_table
<br>Iterate each th element and apply the provided extract_column_from_header() to get a column name
<br>Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

In [ ]:
# Table content
html_tables = soup.find_all('table')
# 3rd table content
first_launch_table = html_tables[2]
# print(first_launch_table)

In [ ]:
# Column names
column_names = []
for th in first_launch_table.find_all('th'):
    column_name = extract_column_from_header(th)
    if column_name:  # Append if non-empty
        column_names.append(column_name)

print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


# Data Pre-processing

## Segregrate data

Next, we just need to fill up the launch_dict with launch records extracted from table rows.

Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links B0004.1[8], missing values N/A [e], inconsistent formatting, etc.

To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the launch_dict.

In [ ]:
# Create data structure dictionary
launch_dict= dict.fromkeys(column_names)
# Remove an irrelvant column
del launch_dict['Date and time ( )']

launch_dict = {
    "Flight No.": [],
    "Date": [],
    "Time": [],
    "Version Booster": [],
    "Launch Site": [],
    "Payload": [],
    "Payload mass": [],
    "Orbit": [],
    "Customer": [],
    "Launch outcome": [],
    "Booster landing": []
}

In [ ]:
extracted_row = 0
# Extract each table
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # get table row
    for rows in table.find_all("tr"):
        # check to see if first table heading is a number corresponding to a launch
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False

        # get table element
        row = rows.find_all('td')

        # if it is a number, save cells in a dictionary
        if flag:
            extracted_row += 1

            # Flight Number value
            launch_dict["Flight No."].append(flight_number)

            # Date value
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            launch_dict["Date"].append(date)

            # Time value
            time = datatimelist[1]
            launch_dict["Time"].append(time)

            # Booster version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string
            launch_dict["Version Booster"].append(bv)

            # Launch Site
            launch_site = row[2].a.string
            launch_dict["Launch Site"].append(launch_site)

            # Payload
            payload = row[3].a.string
            launch_dict["Payload"].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict["Payload mass"].append(payload_mass)

            # Orbit
            orbit = row[5].a.string
            launch_dict["Orbit"].append(orbit)

            # Customer
            customer = row[6].a.string
            launch_dict["Customer"].append(customer)

            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict["Launch outcome"].append(launch_outcome)

            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict["Booster landing"].append(booster_landing)

AttributeError: 'NoneType' object has no attribute 'string'

### Reconstruct dataframe

Finally lets construct our dataset using the data we have obtained.
<br>We we combine the columns into a dictionary.


Then, we need to create a Pandas data frame from the dictionary launch_dict.

In [ ]:
# Create final dataframe from dictionary
launch_df = pd.DataFrame({key:pd.Series(value) for key, value in launch_dict.items()})
# launch_df = pd.DataFrame(launch_dict)
launch_df.head()

,Flight No.,Date,Time,Version Booster,Launch Site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,1,4 June 2010,18:45,F9 v1.07B0003.18,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,Failure
1,2,8 December 2010,15:43,F9 v1.07B0004.18,CCAFS,Dragon,0,LEO,NASA,Success,Failure
2,3,22 May 2012,07:44,F9 v1.07B0005.18,CCAFS,Dragon,525 kg,LEO,NASA,Success,No attempt\n
3,4,8 October 2012,00:35,F9 v1.07B0006.18,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,No attempt
4,5,1 March 2013,15:10,F9 v1.07B0007.18,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,No attempt\n


# Export Web Data

We can now export web scrapping data to a CSV file

In [ ]:
# Wikipedia dataframe
launch_df.head()

,Flight No.,Date,Time,Version Booster,Launch Site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,1,4 June 2010,18:45,F9 v1.07B0003.18,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,Failure
1,2,8 December 2010,15:43,F9 v1.07B0004.18,CCAFS,Dragon,0,LEO,NASA,Success,Failure
2,3,22 May 2012,07:44,F9 v1.07B0005.18,CCAFS,Dragon,525 kg,LEO,NASA,Success,No attempt\n
3,4,8 October 2012,00:35,F9 v1.07B0006.18,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,No attempt
4,5,1 March 2013,15:10,F9 v1.07B0007.18,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,No attempt\n


In [ ]:
# Save data as CSV file
launch_df.to_csv('spacex_web_scraped.csv', index=False)